In [2]:
import os
import ast
from dotenv import load_dotenv
import arxiv
import requests

from langchain_openai import ChatOpenAI
from langchain_huggingface import HuggingFaceEndpoint, ChatHuggingFace

from duckduckgo_search import DDGS

load_dotenv()

OPENAI_API_KEY = os.getenv('OPENAI_API_KEY')
HUGGINGFACEHUB_API_TOKEN = os.getenv('HUGGINGFACEHUB_API_TOKEN')
OPENWEATHERMAP_API_KEY = os.getenv('OPENWEATHERMAP_API_KEY')

In [11]:
chat_model = ChatOpenAI(stop_sequences=["Observation:"])

In [4]:
from langchain_core.documents import Document
from langchain_chroma import Chroma

from langchain_huggingface.embeddings import HuggingFaceEmbeddings
from langchain_community.document_loaders import PyPDFLoader
from langchain_text_splitters import RecursiveCharacterTextSplitter

from langchain_openai import ChatOpenAI

chat_model = ChatOpenAI()

file_path = (
    "../../eth_courses/thesis/gokberk_thesis_report_04_06_23.pdf"
)
loader = PyPDFLoader(file_path)
pages = loader.load_and_split()

text_splitter = RecursiveCharacterTextSplitter(
    chunk_size=1000, chunk_overlap=200, add_start_index=True
)
all_splits = text_splitter.split_documents(pages)

vectorstore = Chroma.from_documents(
    all_splits,
    embedding=HuggingFaceEmbeddings(),
)

rag_prompt = """System: You are an assistant for question-answering tasks. 
Use the following pieces of retrieved context to answer the question. If you don't know the answer, just say that you don't know.
Use three sentences maximum and keep the answer concise.

Context: {context}

Human: {input}
AI:
"""

contextualize_q_system_prompt = """Given a chat history and the latest user question which might reference context in the chat history, 
formulate a standalone question which can be understood without the chat history. 
Do NOT answer the question, just reformulate it if needed and otherwise return it as is.
If chat history is empty (''), do NOT change anything in the question.

Chat history:
{history}
Question:
{input}
Reformulated Question:
"""

duck_prompt = """System: You are an assistant for question-answering tasks. 
Use the following pieces of retrieved context to answer the question. If you don't know the answer, just say that you don't know.
Use three sentences maximum and keep the answer concise.

Context: {context}

Human: {input}
AI:
"""

/Users/gokberk/Desktop/self-projects/langchain-from-scratch/chatbotvenv/lib/python3.10/site-packages/sentence_transformers/cross_encoder/CrossEncoder.py:11: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from tqdm.autonotebook import tqdm, trange
/Users/gokberk/Desktop/self-projects/langchain-from-scratch/chatbotvenv/lib/python3.10/site-packages/threadpoolctl.py:1214: RuntimeWarning: 
Found Intel OpenMP ('libiomp') and LLVM OpenMP ('libomp') loaded at
the same time. Both libraries are known to be incompatible and this
can cause random crashes or deadlocks on Linux when loaded in the
same Python program.
Using threadpoolctl may cause crashes or deadlocks. For more
information and possible workarounds, please see
    https://github.com/joblib/threadpoolctl/blob/master/multiple_openmp.md

  warnings.warn(msg, RuntimeWarning)


In [5]:
def search_arxiv(args):
    print('ARXIV')
    # Construct the default API client.
    client = arxiv.Client()

    # Search for the paper with ID
    query = args['query'] if 'query' in args else ""
    id_list = [args['id']] if 'id' in args else []
    
    search_by_id = arxiv.Search(query=query,
                                id_list=id_list)
    # Reuse client to fetch the paper, then print its title.
    first_result = next(client.results(search_by_id))
    
    return first_result.title

def convert_time(args):
    print('CONVERT TIME')
    temp = args['time'].split(':')

    return int(temp[0])*3600 + int(temp[1])*60 + int(temp[2])


def search_frus(args):
    print('FRUS')
    global history

    question = args if not 'question' in args else args['question']

    rephrased_q = chat_model.invoke(contextualize_q_system_prompt.format(input=question,history=history)).content
    print(f"REPHRASED QUESTION: {rephrased_q}")

    context = "\n\n".join([doc.page_content for doc in vectorstore.similarity_search(rephrased_q)])

    prompt_after_formatting = rag_prompt.format(input=rephrased_q,
                                            context=context)
    print(prompt_after_formatting)
    output = chat_model.invoke(prompt_after_formatting)

    return output.content

def search_web(args):
    print('WEB SEARCH')
    question = args if not 'question' in args else args['question']
    
    results = DDGS().text(question, max_results=5)

    context = "\n".join([r['body'] for r in results])

    prompt_after_formatting = duck_prompt.format(input=question,
                                                context=context)

    output = chat_model.invoke(prompt_after_formatting)

    return output.content

def search_weather(args):
    print('WEATHER SEARCH')
    city, country = args['city'], args['country']

    r = requests.get('http://api.openweathermap.org/data/2.5/weather?q='+city+','+country+'&units=metric&APPID='+OPENWEATHERMAP_API_KEY)

    r = ast.literal_eval(r.content.decode())

    res_dict = {
        'description': r['weather'][0]['description'],
        'temperature': r['main']['temp'],
        'city': r['name'],
        'country': r['sys']['country']}
    
    return res_dict

In [6]:
system_prompt = """You are a helpful assistant that can help with a wide range of tasks and provide valuable insights and information on a wide range of topics. 
Whether the human needs help with a specific question or just want to have a conversation about a particular topic, you are here to assist. 
You may not need to use tools for every query - the human may just want to chat!

TOOLS:
------
Assistant has access to the following tools:

{tools}

To use a tool, you MUST use the following format:

Question: the input question you must answer
Thought: Do I need to use a tool? Yes. (only one action at a time.)
Action: the action to take, should be one of {tool_names}
Action Input: the input to the tool (format MUST match tool's indicated args)
Observation: the result of the action
... (this Thought/Action/Action Input/Observation can repeat N times)


When you have a response to say to the Human, or if you do not need to use a tool, you MUST use the format:

Thought: Do I need to use a tool? No.
Final Answer: [your response here]
I remind you to ALWAYS use the exact characters `Final Answer:` when you provide an answer.

Begin!

Conversation History: 
{history}
Question: {input}
Thought:{agent_scratchpad}
"""

tools = {
    'convert_time': 'A function to convert a time string with format H:MM:SS to seconds, args: {"time": {"type": "string"}}',
    'search_arxiv': 'A function to get information about a scientific article or articles, args: {"query": {"type": "string"}, "id": {"type": "string"}}',
    'search_web': 'A function to find an answer for a question using web search, args: {"question": {"type": "string"}}',
    'search_weather': 'A function to get information about weather forecast, args: {"city": {"type": "string"}, "country": {"type": "string"}}',
    'search_frus': 'A function to find an answer for a question on documents about politics of United States of America, args: {"question": {"type": "string"}}'
}

In [12]:
history = ''
agent_scratchpad = ''

def ask(question):

    global history, agent_scratchpad

    tools_str = ''

    for func, desc in tools.items():
        tools_str += f"{func} : {desc}\n"
    
    prompt_after_formatting = system_prompt.format(input=question, 
                                                   agent_scratchpad=agent_scratchpad, 
                                                   tools=tools_str, 
                                                   tool_names=list(tools.keys()),
                                                   history=history)
    print(prompt_after_formatting)
    output = chat_model.invoke(prompt_after_formatting)
    print(output.content)
    
    # keep reasoning
    while output.content.find("Final Answer: ")==-1:

        agent_scratchpad += output.content

        function_name = output.content.split("Action: ")[1].split('\n')[0]
        function_args = ast.literal_eval(output.content.split("Action Input: ")[1].split('\n')[0])

        result = globals()[function_name](function_args)
        agent_scratchpad += f"Observation: {result}\n"
        print(f"Observation: {result}\n")
        
        prompt_after_formatting = system_prompt.format(input=question, 
                                                    agent_scratchpad=agent_scratchpad, 
                                                    tools=tools_str, 
                                                    tool_names=list(tools.keys()),
                                                    history=history)

        output = chat_model.invoke(prompt_after_formatting)
        print(output.content)
    
    # store question and answer in memory
    history += f"Human: {question}\nAI: {output.content.split('Final Answer: ')[1]}\n"
    
    # empty scratchpad for next question
    agent_scratchpad = ''

    return output.content

In [13]:
ask("First answer how many seconds in 4:00:01.")

You are a helpful assistant that can help with a wide range of tasks and provide valuable insights and information on a wide range of topics. 
Whether the human needs help with a specific question or just want to have a conversation about a particular topic, you are here to assist. 
You may not need to use tools for every query - the human may just want to chat!

TOOLS:
------
Assistant has access to the following tools:

convert_time : A function to convert a time string with format H:MM:SS to seconds, args: {"time": {"type": "string"}}
search_arxiv : A function to get information about a scientific article or articles, args: {"query": {"type": "string"}, "id": {"type": "string"}}
search_web : A function to find an answer for a question using web search, args: {"question": {"type": "string"}}
search_weather : A function to get information about weather forecast, args: {"city": {"type": "string"}, "country": {"type": "string"}}
search_frus : A function to find an answer for a question 

'Thought: Do I need to use a tool? No.\nFinal Answer: 14401 seconds in 4:00:01.'

In [14]:
ask("Then answer What's the paper 1605.08386 about?")

You are a helpful assistant that can help with a wide range of tasks and provide valuable insights and information on a wide range of topics. 
Whether the human needs help with a specific question or just want to have a conversation about a particular topic, you are here to assist. 
You may not need to use tools for every query - the human may just want to chat!

TOOLS:
------
Assistant has access to the following tools:

convert_time : A function to convert a time string with format H:MM:SS to seconds, args: {"time": {"type": "string"}}
search_arxiv : A function to get information about a scientific article or articles, args: {"query": {"type": "string"}, "id": {"type": "string"}}
search_web : A function to find an answer for a question using web search, args: {"question": {"type": "string"}}
search_weather : A function to get information about weather forecast, args: {"city": {"type": "string"}, "country": {"type": "string"}}
search_frus : A function to find an answer for a question 

'Thought: Do I need to use a tool? No.\nFinal Answer: Heat-bath random walks with Markov bases.'

In [15]:
ask("What's the previous paper about?")

You are a helpful assistant that can help with a wide range of tasks and provide valuable insights and information on a wide range of topics. 
Whether the human needs help with a specific question or just want to have a conversation about a particular topic, you are here to assist. 
You may not need to use tools for every query - the human may just want to chat!

TOOLS:
------
Assistant has access to the following tools:

convert_time : A function to convert a time string with format H:MM:SS to seconds, args: {"time": {"type": "string"}}
search_arxiv : A function to get information about a scientific article or articles, args: {"query": {"type": "string"}, "id": {"type": "string"}}
search_web : A function to find an answer for a question using web search, args: {"question": {"type": "string"}}
search_weather : A function to get information about weather forecast, args: {"city": {"type": "string"}, "country": {"type": "string"}}
search_frus : A function to find an answer for a question 

'Thought: Do I need to use a tool? No.\nFinal Answer: The previous paper is about heat-bath random walks with Markov bases.'

In [16]:
ask("how is weather in balikesir, tr right now?")

You are a helpful assistant that can help with a wide range of tasks and provide valuable insights and information on a wide range of topics. 
Whether the human needs help with a specific question or just want to have a conversation about a particular topic, you are here to assist. 
You may not need to use tools for every query - the human may just want to chat!

TOOLS:
------
Assistant has access to the following tools:

convert_time : A function to convert a time string with format H:MM:SS to seconds, args: {"time": {"type": "string"}}
search_arxiv : A function to get information about a scientific article or articles, args: {"query": {"type": "string"}, "id": {"type": "string"}}
search_web : A function to find an answer for a question using web search, args: {"question": {"type": "string"}}
search_weather : A function to get information about weather forecast, args: {"city": {"type": "string"}, "country": {"type": "string"}}
search_frus : A function to find an answer for a question 

'Thought: Do I need to use a tool? No.\nFinal Answer: The weather in Balıkesir, Turkey right now is clear sky with a temperature of 27.98°C.'

In [17]:
ask("what did i ask you recently?")

You are a helpful assistant that can help with a wide range of tasks and provide valuable insights and information on a wide range of topics. 
Whether the human needs help with a specific question or just want to have a conversation about a particular topic, you are here to assist. 
You may not need to use tools for every query - the human may just want to chat!

TOOLS:
------
Assistant has access to the following tools:

convert_time : A function to convert a time string with format H:MM:SS to seconds, args: {"time": {"type": "string"}}
search_arxiv : A function to get information about a scientific article or articles, args: {"query": {"type": "string"}, "id": {"type": "string"}}
search_web : A function to find an answer for a question using web search, args: {"question": {"type": "string"}}
search_weather : A function to get information about weather forecast, args: {"city": {"type": "string"}, "country": {"type": "string"}}
search_frus : A function to find an answer for a question 

'Do I need to use a tool? No.\nFinal Answer: You recently asked me how is the weather in Balıkesir, Turkey right now.'

In [18]:
ask("can you search web about object oriented programming and give me information about it?")

You are a helpful assistant that can help with a wide range of tasks and provide valuable insights and information on a wide range of topics. 
Whether the human needs help with a specific question or just want to have a conversation about a particular topic, you are here to assist. 
You may not need to use tools for every query - the human may just want to chat!

TOOLS:
------
Assistant has access to the following tools:

convert_time : A function to convert a time string with format H:MM:SS to seconds, args: {"time": {"type": "string"}}
search_arxiv : A function to get information about a scientific article or articles, args: {"query": {"type": "string"}, "id": {"type": "string"}}
search_web : A function to find an answer for a question using web search, args: {"question": {"type": "string"}}
search_weather : A function to get information about weather forecast, args: {"city": {"type": "string"}, "country": {"type": "string"}}
search_frus : A function to find an answer for a question 

'Thought: Do I need to use a tool? No.\nFinal Answer: Object-oriented programming (OOP) is a programming paradigm based on objects containing data and code. OOP focuses on defining classes with attributes and behaviors. It uses classes and objects to structure software.'

In [19]:
ask("who is kissinger?")

You are a helpful assistant that can help with a wide range of tasks and provide valuable insights and information on a wide range of topics. 
Whether the human needs help with a specific question or just want to have a conversation about a particular topic, you are here to assist. 
You may not need to use tools for every query - the human may just want to chat!

TOOLS:
------
Assistant has access to the following tools:

convert_time : A function to convert a time string with format H:MM:SS to seconds, args: {"time": {"type": "string"}}
search_arxiv : A function to get information about a scientific article or articles, args: {"query": {"type": "string"}, "id": {"type": "string"}}
search_web : A function to find an answer for a question using web search, args: {"question": {"type": "string"}}
search_weather : A function to get information about weather forecast, args: {"city": {"type": "string"}, "country": {"type": "string"}}
search_frus : A function to find an answer for a question 

'Thought: Do I need to use a tool? No.\nFinal Answer: Henry Alfred Kissinger was an American diplomat and political scientist who served as the United States Secretary of State and National Security Advisor. He was awarded the Nobel Peace Prize in 1973.'

In [20]:
ask("what are the most redacted topics in frus?")

You are a helpful assistant that can help with a wide range of tasks and provide valuable insights and information on a wide range of topics. 
Whether the human needs help with a specific question or just want to have a conversation about a particular topic, you are here to assist. 
You may not need to use tools for every query - the human may just want to chat!

TOOLS:
------
Assistant has access to the following tools:

convert_time : A function to convert a time string with format H:MM:SS to seconds, args: {"time": {"type": "string"}}
search_arxiv : A function to get information about a scientific article or articles, args: {"query": {"type": "string"}, "id": {"type": "string"}}
search_web : A function to find an answer for a question using web search, args: {"question": {"type": "string"}}
search_weather : A function to get information about weather forecast, args: {"city": {"type": "string"}, "country": {"type": "string"}}
search_frus : A function to find an answer for a question 

'Thought: Do I need to use a tool? No.\nFinal Answer: The most redacted topics in FRUS include military, intelligence, war, and economical aspects.'

In [22]:
ask("was he redacted in frus?")

You are a helpful assistant that can help with a wide range of tasks and provide valuable insights and information on a wide range of topics. 
Whether the human needs help with a specific question or just want to have a conversation about a particular topic, you are here to assist. 
You may not need to use tools for every query - the human may just want to chat!

TOOLS:
------
Assistant has access to the following tools:

convert_time : A function to convert a time string with format H:MM:SS to seconds, args: {"time": {"type": "string"}}
search_arxiv : A function to get information about a scientific article or articles, args: {"query": {"type": "string"}, "id": {"type": "string"}}
search_web : A function to find an answer for a question using web search, args: {"question": {"type": "string"}}
search_weather : A function to get information about weather forecast, args: {"city": {"type": "string"}, "country": {"type": "string"}}
search_frus : A function to find an answer for a question 

'Thought: Do I need to use a tool? No.\nFinal Answer: Yes, Henry Alfred Kissinger is mentioned in FRUS but not redacted.'

In [37]:
ask("fenerbahce son resmi maci skoru?")

You are a helpful assistant that can help with a wide range of tasks and provide valuable insights and information on a wide range of topics. 
Whether the human needs help with a specific question or just want to have a conversation about a particular topic, you are here to assist. 
You may not need to use tools for every query - the human may just want to chat!

TOOLS:
------
Assistant has access to the following tools:

convert_time : A function to convert a time string with format H:MM:SS to seconds, args: {"time": {"type": "string"}}
search_arxiv : A function to get information about a scientific article or articles, args: {"query": {"type": "string"}, "id": {"type": "string"}}
search_web : A function to find an answer for a question using web search, args: {"question": {"type": "string"}}
search_weather : A function to get information about weather forecast, args: {"city": {"type": "string"}, "country": {"type": "string"}}
search_frus : A function to find an answer for a question 

'Thought: Do I need to use a tool? No.\nFinal Answer: Fenerbahçe won their last official match with a score of 6 - 0 against İstanbulspor in the Trendyol Süper Lig.'